# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

# loading the dataframe with all the information that saved in previous task
source_df=pd.read_csv('../output_data/City_Coord_data.csv')
source_df

,Unnamed: 0,name,country,lon,lat,humidity,temp_max,wind_speed,cloudness
0,0,Yellowknife,CA,-114.3525,62.4560,85,-17.59,4.00,100
1,1,Barrow,US,-156.7887,71.2906,69,-16.58,0.00,1
2,2,Hobart,AU,147.3294,-42.8794,70,62.02,1.01,75
3,3,Vila Velha,BR,-40.2925,-20.3297,97,73.94,7.43,100
4,4,East London,ZA,27.9116,-33.0153,77,75.51,6.31,100
...,...,...,...,...,...,...,...,...,...
539,539,Huarmey,PE,-78.1522,-10.0681,83,70.25,7.11,15
540,540,Mabaruma,GY,-59.7833,8.2000,96,69.06,2.37,98
541,541,Philadelphia,US,-75.1638,39.9523,65,38.19,5.99,0
542,542,Bac Lieu,VN,105.7244,9.2850,61,84.54,19.13,70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key

gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = source_df[["lat", "lon"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = source_df["humidity"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure(center=(0,0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=source_df["humidity"].max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
#print(source_df.info())
narrow_data=source_df.loc[((source_df['wind_speed']<10)&
                           (source_df['temp_max']>70)&
                           (source_df['temp_max']<80)&
                          (source_df['cloudness']!=0))]
narrow_data=narrow_data.dropna(subset=['country','wind_speed','temp_max','cloudness'])
print(narrow_data)

     Unnamed: 0          name country       lon      lat  humidity  temp_max  \
3             3    Vila Velha      BR  -40.2925 -20.3297        97     73.94   
4             4   East London      ZA   27.9116 -33.0153        77     75.51   
23           23   Port Alfred      ZA   26.8910 -33.5906        67     77.47   
30           30          Hilo      US -155.0900  19.7297        98     72.25   
41           41     Sinnamary      GF  -52.9500   5.3833        89     74.30   
..          ...           ...     ...       ...      ...       ...       ...   
495         495        Jimeta      NG   12.4667   9.2833        21     74.14   
511         511       Chazuta      PE  -76.1367  -6.5742        99     72.25   
512         512          Tura      IN   90.2201  25.5198        38     77.18   
526         526  Nova Olímpia      BR  -57.2881 -14.7972        96     71.65   
539         539       Huarmey      PE  -78.1522 -10.0681        83     70.25   

     wind_speed  cloudness  
3         

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
import pprint as pprint
import requests
import json

hotel_df = pd.DataFrame(columns=['Hotel Name','City','Country','lat','lng'])
#set the number of cities i have set 20 out of 81
city_number=10
for i in range(city_number):
            # geocoordinates
    target_coordinates = f'{narrow_data.iloc[i,4]}, {narrow_data.iloc[i,3]}'
    target_search ="hotel"
    target_radius = 5000
    target_type = "hotel"
  

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
 
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
            #append to hotel data frame
        hotel_df=hotel_df.append({'Hotel Name':results[0]["name"],
                              'City':narrow_data.iloc[i,1],
                              'Country':narrow_data.iloc[i,2],
                              'lat':results[0]["geometry"]["location"]["lat"],
                              'lng':results[0]["geometry"]["location"]["lng"]}, ignore_index=True)
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    


# Print the hotel information data frae
hotel_df

,Hotel Name,City,Country,lat,lng
0,Quality Suites Vila Velha,Vila Velha,BR,-20.335028,-40.280975
1,Blue Lagoon Hotel,East London,ZA,-32.983472,27.951098
2,Royal St Andrews Hotel,Port Alfred,ZA,-33.603178,26.886978
3,Hilo Hawaiian Hotel,Hilo,US,19.728404,-155.066467
4,Hotel Du Fleuve,Sinnamary,GF,5.361880,-52.948137
5,The Grand Hotel,Port Elizabeth,ZA,-33.963425,25.619788
6,Paradise First Hotel,Vaini,TO,-21.155314,-175.177150
7,Hotel Porto do Sol,Guarapari,BR,-20.662936,-40.495522
8,Hotel Du Lac,Inongo,CD,-1.946559,18.266514
9,Hotel Villas Del Mar,Pedernales,DO,18.029441,-71.743419


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [48]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))